### Extension: SOM quantization error

Do an analysis (make plots, explain findings) where you investigate how various SOM parameters affect the quantization error. Parameters you might experiment with are:
- iteration count
- SOM size

**NOTE:** Some parameters may need to adjusted together in some fashion. For example, decreasing the SOM size decreases the grid size. Therefore, the Gaussian neighborhood size $\sigma$ probably needs to be rescaled, as does the learning rate (how much vectors move around in the space duing each update).

## Overall Summary
In this project we try a bunch of different SOM Sizes, then Learning Rates with and without those SOM sizes, and finally adjustments to Sigma, with and without the SOM Sizes and Learning Rates. We attempt to see what changes cause what and where we can make improvements.

We find that overall, SOM Size at the larger level provides the lowest final Quantization error. I hypothesize this is because there is more to learn with a larger SOM size. For example, when we use 500, we have a final error of .1 while using size 7 gives us about .03. Over time, there can be more finegrained stuff we learn with a larger grid size and make the distance between nodes closer. 

Learning rate is also important. Learning Rate causes the most changes in the first iteration. This is because it changes the distance that different nodes can move. If we have a large grid and the nodes can only move a small distance, this would increase error for the first iteration. This is because a small change would not consitute the distance needed to travel. So, increasing error rate helps our first iteration have lower error unless we make learning rate too high and the node moves too much. However, over time as more iterations are done, learning rate decays and we do not view and changes to later errors. Overall, high LR = More first iteration accuracy and a low LR has high first iteration error. 

Sigma has little changes to add. We see a slight change in the range Sigma [7, 15]. Overall though, outside that range, which only creates a first iteration change of [.14, .16], we view no change. Therefore, Sigma 15 and Sigma 500 cause no difference. Same thing with Sigma 7 and Sigma .0005. This is interesting because it probably has to do with the Sigma Decay. 

Overall this extension was super interesting. I thought Learning Rate would cause bigger adjustments than it did and mapping this all in my head to movement on euclidean axes was helpful. I understand what the hyperparameters do significantly more now.

In [92]:
import os
import random
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

plt.style.use(['seaborn-colorblind', 'seaborn-darkgrid'])
plt.rcParams.update({'font.size': 20})

np.set_printoptions(suppress=True, precision=3)

# Automatically reload external modules
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [93]:
import som

In [94]:
iris = pd.read_csv('https://raw.githubusercontent.com/mwaskom/seaborn-data/master/iris.csv')
iris.head()
# make iris_x
iris_x = iris.copy()
iris_x = iris_x.drop(columns='species')
iris_x = iris_x.to_numpy()
for i in range(iris_x.shape[0]):
    iris_x[i] = iris_x[i] / np.linalg.norm(iris_x[i])

In [95]:
# make iris_y
iris_y = iris['species']
# print(iris_y)
int_coded = pd.factorize(iris_y)
# print(int_coded)
iris_y = int_coded[0]
print(iris_y)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]


In [9]:
som_sz = 7
n_features = 4
max_iter = 1

np.random.seed(0)

iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_u_map0 = iris_som.u_matrix()
iris_som.fit(iris_x.copy())
iris_u_map = iris_som.u_matrix()
print(f'Quantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')
print(f'Your initial u-matrix is (before any training):\n{iris_u_map0}')
print(f'Your u-matrix is (after 1 step):\n{iris_u_map}\n')

max_iter = 100
iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_som.fit(iris_x.copy())
iris_u_map2 = iris_som.u_matrix()
print(f'Quantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')
print(f'Your u-matrix is (after 100 steps):\n{iris_u_map2}')

max_iter = 1000
iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_som.fit(iris_x.copy())
print(f'\nQuantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')



Quantization error after 1 iterations: 0.15
Your initial u-matrix is (before any training):
[[0.268 0.495 0.516 0.496 0.635 0.582 0.39 ]
 [0.538 0.956 0.931 0.794 0.776 0.771 0.464]
 [0.506 0.911 0.831 0.988 1.    0.751 0.502]
 [0.524 0.771 0.748 0.85  0.879 0.886 0.618]
 [0.405 0.745 0.755 0.714 0.782 0.907 0.586]
 [0.586 0.798 0.733 0.761 0.801 0.959 0.494]
 [0.311 0.508 0.44  0.409 0.48  0.548 0.417]]
Your u-matrix is (after 1 step):
[[0.267 0.494 0.516 0.498 0.643 0.595 0.4  ]
 [0.532 0.946 0.924 0.794 0.782 0.783 0.473]
 [0.497 0.894 0.819 0.98  1.    0.755 0.51 ]
 [0.511 0.753 0.732 0.837 0.872 0.885 0.621]
 [0.392 0.721 0.734 0.698 0.771 0.902 0.587]
 [0.564 0.771 0.71  0.741 0.787 0.951 0.495]
 [0.3   0.49  0.425 0.398 0.471 0.543 0.416]]

Quantization error after 100 iterations: 0.07
Your u-matrix is (after 100 steps):
[[0.131 0.271 0.289 0.261 0.217 0.147 0.066]
 [0.348 0.549 0.529 0.479 0.386 0.285 0.137]
 [0.518 0.774 0.719 0.63  0.518 0.398 0.218]
 [0.634 0.971 0.913 0.797

# Plot 0:
Above is out first, normal plot from the regular notebook. We will use this as a baseline.
It has an error output of [.15, .07, .03].

## SOM Size Changes
### Decreases
In these next several code blocks, we will be messing around with SOM size. First we will decrease then eventually increase SOM size. Learning Rate and Sigma will not be touched.

In [11]:
som_sz = 5
n_features = 4
max_iter = 1

np.random.seed(0)

iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_u_map0 = iris_som.u_matrix()
iris_som.fit(iris_x.copy())
iris_u_map = iris_som.u_matrix()
print(f'Quantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')
print(f'Your initial u-matrix is (before any training):\n{iris_u_map0}')
print(f'Your u-matrix is (after 1 step):\n{iris_u_map}\n')

max_iter = 100
iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_som.fit(iris_x.copy())
iris_u_map2 = iris_som.u_matrix()
print(f'Quantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')
print(f'Your u-matrix is (after 100 steps):\n{iris_u_map2}')

max_iter = 1000
iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_som.fit(iris_x.copy())
print(f'\nQuantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')

Quantization error after 1 iterations: 0.22
Your initial u-matrix is (before any training):
[[0.291 0.445 0.519 0.41  0.259]
 [0.636 0.829 0.868 0.863 0.505]
 [0.411 0.68  0.768 0.742 0.542]
 [0.502 0.758 0.84  1.    0.537]
 [0.294 0.574 0.5   0.463 0.303]]
Your u-matrix is (after 1 step):
[[0.29  0.442 0.514 0.406 0.257]
 [0.636 0.826 0.862 0.855 0.501]
 [0.413 0.679 0.764 0.738 0.539]
 [0.506 0.761 0.84  1.    0.537]
 [0.298 0.579 0.503 0.464 0.304]]

Quantization error after 100 iterations: 0.08
Your u-matrix is (after 100 steps):
[[0.304 0.5   0.606 0.554 0.23 ]
 [0.51  0.922 0.997 0.916 0.538]
 [0.617 1.    0.973 0.811 0.444]
 [0.56  0.891 0.8   0.595 0.296]
 [0.217 0.519 0.433 0.292 0.155]]

Quantization error after 1000 iterations: 0.04


### Plot 1: Decreasing SOM size to 5
When we decrease out SOM, we get a higher error rate. This is only decreasing to SOM: 5 but it has a [.22,.08,.04] output, higher across the board.

In [14]:
som_sz = 3
n_features = 4
max_iter = 1

np.random.seed(0)

iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_u_map0 = iris_som.u_matrix()
iris_som.fit(iris_x.copy())
iris_u_map = iris_som.u_matrix()
print(f'Quantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')
print(f'Your initial u-matrix is (before any training):\n{iris_u_map0}')
print(f'Your u-matrix is (after 1 step):\n{iris_u_map}\n')

max_iter = 100
iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_som.fit(iris_x.copy())
iris_u_map2 = iris_som.u_matrix()
print(f'Quantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')
print(f'Your u-matrix is (after 100 steps):\n{iris_u_map2}')

max_iter = 1000
iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_som.fit(iris_x.copy())
print(f'\nQuantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')

Quantization error after 1 iterations: 0.23
Your initial u-matrix is (before any training):
[[0.287 0.588 0.498]
 [0.385 1.    0.853]
 [0.289 0.697 0.512]]
Your u-matrix is (after 1 step):
[[0.288 0.589 0.499]
 [0.385 1.    0.853]
 [0.289 0.696 0.511]]

Quantization error after 100 iterations: 0.13
Your u-matrix is (after 100 steps):
[[0.264 0.645 0.484]
 [0.6   1.    0.645]
 [0.414 0.594 0.267]]

Quantization error after 1000 iterations: 0.09


### Plot 2: Decreasing SOM size to 3
SOM Size = 3 is again causing us a higher error rate. More significantly now. Our grid size is getting small.

In [15]:
som_sz = 2
n_features = 4
max_iter = 1

np.random.seed(0)

iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_u_map0 = iris_som.u_matrix()
iris_som.fit(iris_x.copy())
iris_u_map = iris_som.u_matrix()
print(f'Quantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')
print(f'Your initial u-matrix is (before any training):\n{iris_u_map0}')
print(f'Your u-matrix is (after 1 step):\n{iris_u_map}\n')

max_iter = 100
iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_som.fit(iris_x.copy())
iris_u_map2 = iris_som.u_matrix()
print(f'Quantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')
print(f'Your u-matrix is (after 100 steps):\n{iris_u_map2}')

max_iter = 1000
iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_som.fit(iris_x.copy())
print(f'\nQuantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')

Quantization error after 1 iterations: 0.34
Your initial u-matrix is (before any training):
[[0.647 0.951]
 [1.    0.672]]
Your u-matrix is (after 1 step):
[[0.647 0.951]
 [1.    0.672]]

Quantization error after 100 iterations: 0.17
Your u-matrix is (after 100 steps):
[[0.498 1.   ]
 [0.969 0.498]]

Quantization error after 1000 iterations: 0.16


### Plot 3: Decreasing SOM size to 2
Now our SOM size is at 2. This is small enough to cause large errors. This is obviously more inefficient than a larger SOM size.

In [18]:
som_sz = 9
n_features = 4
max_iter = 1

np.random.seed(0)

iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_u_map0 = iris_som.u_matrix()
iris_som.fit(iris_x.copy())
iris_u_map = iris_som.u_matrix()
print(f'Quantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')
print(f'Your initial u-matrix is (before any training):\n{iris_u_map0}')
print(f'Your u-matrix is (after 1 step):\n{iris_u_map}\n')

max_iter = 100
iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_som.fit(iris_x.copy())
iris_u_map2 = iris_som.u_matrix()
print(f'Quantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')
print(f'Your u-matrix is (after 100 steps):\n{iris_u_map2}')

max_iter = 1000
iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_som.fit(iris_x.copy())
print(f'\nQuantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')

Quantization error after 1 iterations: 0.14
Your initial u-matrix is (before any training):
[[0.28  0.562 0.482 0.572 0.598 0.604 0.583 0.636 0.343]
 [0.59  0.805 0.784 0.852 0.981 0.868 0.915 0.854 0.617]
 [0.617 0.909 0.97  1.    0.734 0.791 0.837 0.973 0.555]
 [0.554 0.911 0.898 0.864 0.8   0.784 0.9   0.999 0.582]
 [0.536 0.891 0.882 0.938 0.92  0.997 0.838 0.832 0.557]
 [0.586 0.88  0.889 0.936 0.909 0.833 0.782 0.859 0.541]
 [0.619 0.679 0.645 0.782 0.952 0.823 0.955 0.802 0.578]
 [0.521 0.814 0.618 0.715 0.893 0.869 0.76  0.888 0.583]
 [0.386 0.545 0.547 0.457 0.529 0.547 0.559 0.515 0.353]]
Your u-matrix is (after 1 step):
[[0.288 0.574 0.489 0.576 0.599 0.604 0.582 0.636 0.344]
 [0.605 0.82  0.793 0.855 0.979 0.865 0.909 0.85  0.616]
 [0.631 0.924 0.978 1.    0.73  0.784 0.828 0.963 0.55 ]
 [0.563 0.922 0.903 0.86  0.791 0.773 0.886 0.986 0.575]
 [0.545 0.902 0.885 0.933 0.91  0.981 0.825 0.82  0.55 ]
 [0.598 0.89  0.892 0.933 0.901 0.822 0.77  0.848 0.535]
 [0.632 0.69  0.651

## Increasing SOM Size

### Plot 4: Increasing SOM size to 9
Our first cell had error of  [.15, .07, .03]. We see here an error of [.14,.05,.03] This is a small decrease and could be boiled down to randomness.


In [20]:
som_sz = 11
n_features = 4
max_iter = 1

np.random.seed(0)

iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_u_map0 = iris_som.u_matrix()
iris_som.fit(iris_x.copy())
iris_u_map = iris_som.u_matrix()
print(f'Quantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')
print(f'Your initial u-matrix is (before any training):\n{iris_u_map0}')
print(f'Your u-matrix is (after 1 step):\n{iris_u_map}\n')

max_iter = 100
iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_som.fit(iris_x.copy())
iris_u_map2 = iris_som.u_matrix()
print(f'Quantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')
print(f'Your u-matrix is (after 100 steps):\n{iris_u_map2}')

max_iter = 1000
iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_som.fit(iris_x.copy())
print(f'\nQuantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')

Quantization error after 1 iterations: 0.14
Your initial u-matrix is (before any training):
[[0.314 0.563 0.519 0.537 0.515 0.528 0.5   0.619 0.534 0.572 0.401]
 [0.482 0.785 0.887 0.808 0.829 0.918 0.871 0.791 0.853 0.957 0.504]
 [0.397 0.779 0.792 0.872 0.852 0.96  0.862 0.854 0.808 0.808 0.459]
 [0.475 0.96  0.902 0.824 0.847 0.881 0.916 0.842 1.    0.764 0.432]
 [0.364 0.738 0.824 0.793 0.81  0.84  0.736 0.774 0.779 0.848 0.608]
 [0.376 0.704 0.666 0.674 0.855 0.909 0.722 0.809 0.799 0.829 0.477]
 [0.368 0.865 0.943 0.682 0.695 0.739 0.916 0.773 0.852 0.853 0.436]
 [0.597 0.778 0.741 0.812 0.809 0.99  0.719 0.868 0.805 0.847 0.432]
 [0.582 0.764 0.979 0.917 0.795 0.763 0.821 0.696 0.71  0.776 0.526]
 [0.586 0.74  0.815 0.821 0.792 0.809 0.794 0.796 0.749 0.818 0.407]
 [0.296 0.601 0.475 0.485 0.439 0.507 0.504 0.465 0.577 0.465 0.281]]
Your u-matrix is (after 1 step):
[[0.321 0.572 0.522 0.533 0.505 0.515 0.484 0.594 0.512 0.547 0.384]
 [0.495 0.797 0.89  0.801 0.811 0.891 0.839 0.

### Plot 5: Increasing SOM size to 11
Considering more decrease in error, especially after 1000 iterations, we can start to see that increasing SOM size might be quite useful. Our initial error is not decreasing however.

In [28]:
som_sz = 20
n_features = 4
max_iter = 1

np.random.seed(0)

iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_u_map = iris_som.u_matrix()
iris_som.fit(iris_x.copy())
iris_u_map = iris_som.u_matrix()
print(f'Quantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')
# print(f'Your initial u-matrix is (before any training):\n{iris_u_map0}')
# print(f'Your u-matrix is (after 1 step):\n{iris_u_map}\n')

max_iter = 100
iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_som.fit(iris_x.copy())
iris_u_map2 = iris_som.u_matrix()
print(f'Quantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')
# print(f'Your u-matrix is (after 100 steps):\n{iris_u_map2}')

max_iter = 1000
iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_som.fit(iris_x.copy())
print(f'\nQuantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')

Quantization error after 1 iterations: 0.15
Quantization error after 100 iterations: 0.03

Quantization error after 1000 iterations: 0.02


### Plot 6: Increasing SOM size to 20
This plot does not have much change except maybe that it is important to view the difference in error in the first iteration.

In [25]:
som_sz = 25
n_features = 4
max_iter = 1

np.random.seed(0)

iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_u_map0 = iris_som.u_matrix()
iris_som.fit(iris_x.copy())
iris_u_map = iris_som.u_matrix()
print(f'Quantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')
# print(f'Your initial u-matrix is (before any training):\n{iris_u_map0}')
# print(f'Your u-matrix is (after 1 step):\n{iris_u_map}\n')

max_iter = 100
iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_som.fit(iris_x.copy())
iris_u_map2 = iris_som.u_matrix()
print(f'Quantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')
# print(f'Your u-matrix is (after 100 steps):\n{iris_u_map2}')

max_iter = 1000
iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_som.fit(iris_x.copy())
print(f'\nQuantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')

Quantization error after 1 iterations: 0.16
Quantization error after 100 iterations: 0.03

Quantization error after 1000 iterations: 0.02


### Plot 7: Increasing SOM size to 25
Here we see another increase in error in the first iteration. This could be because with a larger grid size, the initial values are spaced further apart, increasing error.

In [26]:
som_sz = 50
n_features = 4
max_iter = 1

np.random.seed(0)

iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_u_map0 = iris_som.u_matrix()
iris_som.fit(iris_x.copy())
iris_u_map = iris_som.u_matrix()
print(f'Quantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')
# print(f'Your initial u-matrix is (before any training):\n{iris_u_map0}')
# print(f'Your u-matrix is (after 1 step):\n{iris_u_map}\n')

max_iter = 100
iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_som.fit(iris_x.copy())
iris_u_map2 = iris_som.u_matrix()
print(f'Quantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')
# print(f'Your u-matrix is (after 100 steps):\n{iris_u_map2}')

max_iter = 1000
iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_som.fit(iris_x.copy())
print(f'\nQuantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')

Quantization error after 1 iterations: 0.15
Quantization error after 100 iterations: 0.03

Quantization error after 1000 iterations: 0.01


### Plot 8: Increasing SOM size to 50
A SOM Size of 50 nets us a very low final error. Our initial error does not increase as we hypothesized above.

In [27]:
som_sz = 500
n_features = 4
max_iter = 1

np.random.seed(0)

iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_u_map0 = iris_som.u_matrix()
iris_som.fit(iris_x.copy())
iris_u_map = iris_som.u_matrix()
print(f'Quantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')
# print(f'Your initial u-matrix is (before any training):\n{iris_u_map0}')
# print(f'Your u-matrix is (after 1 step):\n{iris_u_map}\n')

max_iter = 100
iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_som.fit(iris_x.copy())
iris_u_map2 = iris_som.u_matrix()
print(f'Quantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')
# print(f'Your u-matrix is (after 100 steps):\n{iris_u_map2}')

max_iter = 1000
iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_som.fit(iris_x.copy())
print(f'\nQuantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')

Quantization error after 1 iterations: 0.02
Quantization error after 100 iterations: 0.02

Quantization error after 1000 iterations: 0.01


### Plot 9: Increasing SOM size to 500
This is the big kahuna, what happens if we blow the SOM size out of the water to 500. We result in a low initial error, secondary error and final error. This is interesting because maybe there is overall more information being learned or space for clustering causing a decrease in error. 

## Decreasing Learning Rate with SOM Changes
What happens if we change the learning rate to reflect our SOM changes? We will explore below.

In [30]:
som_sz = 5
n_features = 4
max_iter = 1

np.random.seed(0)

iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.15, init_sigma=10.0, verbose=False)
iris_u_map0 = iris_som.u_matrix()
iris_som.fit(iris_x.copy())
iris_u_map = iris_som.u_matrix()
print(f'Quantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')
# print(f'Your initial u-matrix is (before any training):\n{iris_u_map0}')
# print(f'Your u-matrix is (after 1 step):\n{iris_u_map}\n')

max_iter = 100
iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_som.fit(iris_x.copy())
iris_u_map2 = iris_som.u_matrix()
print(f'Quantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')
# print(f'Your u-matrix is (after 100 steps):\n{iris_u_map2}')

max_iter = 1000
iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_som.fit(iris_x.copy())
print(f'\nQuantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')

Quantization error after 1 iterations: 0.24
Quantization error after 100 iterations: 0.08

Quantization error after 1000 iterations: 0.04


### Plot 10: Decreasing SOM size to 5 and LR to .15
Here we see an increase in the error here. Compared to the standard SOM 5 above,  [.22,.08,.04], we have a slight overall error increase in the first iteration, though that could be random, and no change in the others.

In [31]:
som_sz = 3
n_features = 4
max_iter = 1

np.random.seed(0)

iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.1, init_sigma=10.0, verbose=False)
iris_u_map0 = iris_som.u_matrix()
iris_som.fit(iris_x.copy())
iris_u_map = iris_som.u_matrix()
print(f'Quantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')
# print(f'Your initial u-matrix is (before any training):\n{iris_u_map0}')
# print(f'Your u-matrix is (after 1 step):\n{iris_u_map}\n')

max_iter = 100
iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_som.fit(iris_x.copy())
iris_u_map2 = iris_som.u_matrix()
print(f'Quantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')
# print(f'Your u-matrix is (after 100 steps):\n{iris_u_map2}')

max_iter = 1000
iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_som.fit(iris_x.copy())
print(f'\nQuantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')

Quantization error after 1 iterations: 0.25
Quantization error after 100 iterations: 0.13

Quantization error after 1000 iterations: 0.09


### Plot 11: Decreasing SOM size to 3 and LR to .1
Again , we increase in our first iteration. Above, our SOM size of 3 and LR of .2 gave us: [.23, .13, .09]. Learning rate isn't helping us but not hurting us either.

In [32]:
som_sz = 2
n_features = 4
max_iter = 1

np.random.seed(0)

iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.08, init_sigma=10.0, verbose=False)
iris_u_map0 = iris_som.u_matrix()
iris_som.fit(iris_x.copy())
iris_u_map = iris_som.u_matrix()
print(f'Quantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')
# print(f'Your initial u-matrix is (before any training):\n{iris_u_map0}')
# print(f'Your u-matrix is (after 1 step):\n{iris_u_map}\n')

max_iter = 100
iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_som.fit(iris_x.copy())
iris_u_map2 = iris_som.u_matrix()
print(f'Quantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')
# print(f'Your u-matrix is (after 100 steps):\n{iris_u_map2}')

max_iter = 1000
iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_som.fit(iris_x.copy())
print(f'\nQuantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')

Quantization error after 1 iterations: 0.37
Quantization error after 100 iterations: 0.17

Quantization error after 1000 iterations: 0.16


### Plot 12: Decreasing SOM size to 2 and LR to .08
Here we have an increase in error in the first iteration. However, we have no change in error in the second and third sets of iterations. Above we had: [.34, .17, .16].

In [33]:
som_sz = 2
n_features = 4
max_iter = 1

np.random.seed(0)

iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.02, init_sigma=10.0, verbose=False)
iris_u_map0 = iris_som.u_matrix()
iris_som.fit(iris_x.copy())
iris_u_map = iris_som.u_matrix()
print(f'Quantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')
# print(f'Your initial u-matrix is (before any training):\n{iris_u_map0}')
# print(f'Your u-matrix is (after 1 step):\n{iris_u_map}\n')

max_iter = 100
iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_som.fit(iris_x.copy())
iris_u_map2 = iris_som.u_matrix()
print(f'Quantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')
# print(f'Your u-matrix is (after 100 steps):\n{iris_u_map2}')

max_iter = 1000
iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_som.fit(iris_x.copy())
print(f'\nQuantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')

Quantization error after 1 iterations: 0.38
Quantization error after 100 iterations: 0.17

Quantization error after 1000 iterations: 0.16


### Plot 13: Decreasing SOM size to 2 and LR to .02
Again, we have an increase in the error after 1 iteration. This is likely because the values can't move as far to handle change and therefore do not get close on the first iteration. 

In [34]:
som_sz = 2
n_features = 4
max_iter = 1

np.random.seed(0)

iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.15, init_sigma=10.0, verbose=False)
iris_u_map0 = iris_som.u_matrix()
iris_som.fit(iris_x.copy())
iris_u_map = iris_som.u_matrix()
print(f'Quantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')
# print(f'Your initial u-matrix is (before any training):\n{iris_u_map0}')
# print(f'Your u-matrix is (after 1 step):\n{iris_u_map}\n')

max_iter = 100
iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_som.fit(iris_x.copy())
iris_u_map2 = iris_som.u_matrix()
print(f'Quantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')
# print(f'Your u-matrix is (after 100 steps):\n{iris_u_map2}')

max_iter = 1000
iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_som.fit(iris_x.copy())
print(f'\nQuantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')

Quantization error after 1 iterations: 0.35
Quantization error after 100 iterations: 0.17

Quantization error after 1000 iterations: 0.16


### Plot 14: Decreasing SOM size to 2 and LR to .15
Now, we are increasing our learning rate briefly. We have an increase in accuracy after our first iteration. Again: [.34, .17, .16] was our original, with .2. Do we increase over a Learning Rate of .02 here, yes. It seems there is movement in that first iteration that is heavily influenced by learning rate but at this SOM Size, the Learning Rate has little influence over time.

In [36]:
som_sz = 2
n_features = 4
max_iter = 1

np.random.seed(0)

iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.001, init_sigma=10.0, verbose=False)
iris_u_map0 = iris_som.u_matrix()
iris_som.fit(iris_x.copy())
iris_u_map = iris_som.u_matrix()
print(f'Quantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')
# print(f'Your initial u-matrix is (before any training):\n{iris_u_map0}')
# print(f'Your u-matrix is (after 1 step):\n{iris_u_map}\n')

max_iter = 100
iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_som.fit(iris_x.copy())
iris_u_map2 = iris_som.u_matrix()
print(f'Quantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')
# print(f'Your u-matrix is (after 100 steps):\n{iris_u_map2}')

max_iter = 1000
iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_som.fit(iris_x.copy())
print(f'\nQuantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')

Quantization error after 1 iterations: 0.39
Quantization error after 100 iterations: 0.17

Quantization error after 1000 iterations: 0.16


### Plot 15: Decreasing SOM size to 2 and LR to .001
Just to be thorough, let's heavily decrease our Learning Rate. Again, we see an increase in error after our first iteration. Remember, our first one was [.34, .17, .16] with a learning rate of .2. This is a decent size change in our first iteration but not overall disasterous with decrease at lower levels.

## Increasing Learning Rate With SOM SZ 2.

In [37]:
som_sz = 2
n_features = 4
max_iter = 1

np.random.seed(0)

iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.25, init_sigma=10.0, verbose=False)
iris_u_map0 = iris_som.u_matrix()
iris_som.fit(iris_x.copy())
iris_u_map = iris_som.u_matrix()
print(f'Quantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')
# print(f'Your initial u-matrix is (before any training):\n{iris_u_map0}')
# print(f'Your u-matrix is (after 1 step):\n{iris_u_map}\n')

max_iter = 100
iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_som.fit(iris_x.copy())
iris_u_map2 = iris_som.u_matrix()
print(f'Quantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')
# print(f'Your u-matrix is (after 100 steps):\n{iris_u_map2}')

max_iter = 1000
iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_som.fit(iris_x.copy())
print(f'\nQuantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')

Quantization error after 1 iterations: 0.33
Quantization error after 100 iterations: 0.17

Quantization error after 1000 iterations: 0.16


### Plot 16: Decreasing SOM size to 2 and LR to .25
We increased out learning rate here to beyond .2 and we immdiately see an increase over .2 in our first iteration in error.

## Increasing SOM Size and Learning Rate

In [38]:
som_sz = 9
n_features = 4
max_iter = 1

np.random.seed(0)

iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.25, init_sigma=10.0, verbose=False)
iris_u_map0 = iris_som.u_matrix()
iris_som.fit(iris_x.copy())
iris_u_map = iris_som.u_matrix()
print(f'Quantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')
# print(f'Your initial u-matrix is (before any training):\n{iris_u_map0}')
# print(f'Your u-matrix is (after 1 step):\n{iris_u_map}\n')

max_iter = 100
iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_som.fit(iris_x.copy())
iris_u_map2 = iris_som.u_matrix()
print(f'Quantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')
# print(f'Your u-matrix is (after 100 steps):\n{iris_u_map2}')

max_iter = 1000
iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_som.fit(iris_x.copy())
print(f'\nQuantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')

Quantization error after 1 iterations: 0.13
Quantization error after 100 iterations: 0.05

Quantization error after 1000 iterations: 0.03


### Plot 17: Increasing SOM size to 9 and LR to .25
Here we have a decrease in our first iteration again. This makes sense considering our finidings above. Let's remember this output from our first SOM SZ 9 and LR .2: [.14,.05,.03].

In [40]:
som_sz = 20
n_features = 4
max_iter = 1

np.random.seed(0)

iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.35, init_sigma=10.0, verbose=False)
iris_u_map0 = iris_som.u_matrix()
iris_som.fit(iris_x.copy())
iris_u_map = iris_som.u_matrix()
print(f'Quantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')
# print(f'Your initial u-matrix is (before any training):\n{iris_u_map0}')
# print(f'Your u-matrix is (after 1 step):\n{iris_u_map}\n')

max_iter = 100
iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_som.fit(iris_x.copy())
iris_u_map2 = iris_som.u_matrix()
print(f'Quantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')
# print(f'Your u-matrix is (after 100 steps):\n{iris_u_map2}')

max_iter = 1000
iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_som.fit(iris_x.copy())
print(f'\nQuantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')

Quantization error after 1 iterations: 0.13
Quantization error after 100 iterations: 0.03

Quantization error after 1000 iterations: 0.02


### Plot 18: Increasing SOM size to 20 and LR to .35
Our SOM SZ 20 and LR 2 had [.15, .03, .02]. We see no change again except int he first iteration where higher learning rate would matter more.

In [45]:
som_sz = 500
n_features = 4
max_iter = 1

np.random.seed(0)

iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=2, init_sigma=10.0, verbose=False)
iris_u_map0 = iris_som.u_matrix()
iris_som.fit(iris_x.copy())
iris_u_map = iris_som.u_matrix()
print(f'Quantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')
# print(f'Your initial u-matrix is (before any training):\n{iris_u_map0}')
# print(f'Your u-matrix is (after 1 step):\n{iris_u_map}\n')

max_iter = 100
iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_som.fit(iris_x.copy())
iris_u_map2 = iris_som.u_matrix()
print(f'Quantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')
# print(f'Your u-matrix is (after 100 steps):\n{iris_u_map2}')

max_iter = 1000
iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_som.fit(iris_x.copy())
print(f'\nQuantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')

Quantization error after 1 iterations: 0.02
Quantization error after 100 iterations: 0.02

Quantization error after 1000 iterations: 0.01


### Plot 19: Increasing SOM size to 500 and LR to 2
The Big Kahuna is back! This Learning Rate of 2 is large, the SOM SZ is large, and now our results are good! We actually have no increase in change over the first SOM Size of 500 and LR of .2, but that is okay. Considering the SOM Size and the absurd Learning Rate, we have not got too much difference negatively, or positively.

## Fiddling With Learning Rate at LR 25. Increasing First.

In [46]:
som_sz = 25
n_features = 4
max_iter = 1

np.random.seed(0)

iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.3, init_sigma=10.0, verbose=False)
iris_u_map0 = iris_som.u_matrix()
iris_som.fit(iris_x.copy())
iris_u_map = iris_som.u_matrix()
print(f'Quantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')
# print(f'Your initial u-matrix is (before any training):\n{iris_u_map0}')
# print(f'Your u-matrix is (after 1 step):\n{iris_u_map}\n')

max_iter = 100
iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_som.fit(iris_x.copy())
iris_u_map2 = iris_som.u_matrix()
print(f'Quantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')
# print(f'Your u-matrix is (after 100 steps):\n{iris_u_map2}')

max_iter = 1000
iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_som.fit(iris_x.copy())
print(f'\nQuantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')

Quantization error after 1 iterations: 0.15
Quantization error after 100 iterations: 0.03

Quantization error after 1000 iterations: 0.02


### Plot 20: Increasing SOM size to 25 and LR to .3
We had [.16, .03, .02] in our error cascade earlier with a learning rate of .2. Here we have better results than we did earlier. This is unsurprising and falls right in with what we learned.

In [47]:
som_sz = 25
n_features = 4
max_iter = 1

np.random.seed(0)

iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.4, init_sigma=10.0, verbose=False)
iris_u_map0 = iris_som.u_matrix()
iris_som.fit(iris_x.copy())
iris_u_map = iris_som.u_matrix()
print(f'Quantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')
# print(f'Your initial u-matrix is (before any training):\n{iris_u_map0}')
# print(f'Your u-matrix is (after 1 step):\n{iris_u_map}\n')

max_iter = 100
iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_som.fit(iris_x.copy())
iris_u_map2 = iris_som.u_matrix()
print(f'Quantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')
# print(f'Your u-matrix is (after 100 steps):\n{iris_u_map2}')

max_iter = 1000
iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_som.fit(iris_x.copy())
print(f'\nQuantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')

Quantization error after 1 iterations: 0.14
Quantization error after 100 iterations: 0.03

Quantization error after 1000 iterations: 0.02


### Plot 21: Increasing SOM size to 25 and LR to .4
Again, falling right in line here is the increase in accuracy in our first iteration. At this point, we seem to be beating the dead horse, but we are looking for any decrease in error over time. Again, LR .2 for SOM SZ 25 is: [.16, .03, .02].

In [48]:
som_sz = 25
n_features = 4
max_iter = 1

np.random.seed(0)

iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.5, init_sigma=10.0, verbose=False)
iris_u_map0 = iris_som.u_matrix()
iris_som.fit(iris_x.copy())
iris_u_map = iris_som.u_matrix()
print(f'Quantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')
# print(f'Your initial u-matrix is (before any training):\n{iris_u_map0}')
# print(f'Your u-matrix is (after 1 step):\n{iris_u_map}\n')

max_iter = 100
iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_som.fit(iris_x.copy())
iris_u_map2 = iris_som.u_matrix()
print(f'Quantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')
# print(f'Your u-matrix is (after 100 steps):\n{iris_u_map2}')

max_iter = 1000
iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_som.fit(iris_x.copy())
print(f'\nQuantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')

Quantization error after 1 iterations: 0.13
Quantization error after 100 iterations: 0.03

Quantization error after 1000 iterations: 0.02


### Plot 22: Increasing SOM size to 25 and LR to .5
Here is a decrease in the first iteration again, not much to see or learn here. We are also decreasing error over previous LR's.

In [49]:
som_sz = 25
n_features = 4
max_iter = 1

np.random.seed(0)

iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.7, init_sigma=10.0, verbose=False)
iris_u_map0 = iris_som.u_matrix()
iris_som.fit(iris_x.copy())
iris_u_map = iris_som.u_matrix()
print(f'Quantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')
# print(f'Your initial u-matrix is (before any training):\n{iris_u_map0}')
# print(f'Your u-matrix is (after 1 step):\n{iris_u_map}\n')

max_iter = 100
iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_som.fit(iris_x.copy())
iris_u_map2 = iris_som.u_matrix()
print(f'Quantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')
# print(f'Your u-matrix is (after 100 steps):\n{iris_u_map2}')

max_iter = 1000
iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_som.fit(iris_x.copy())
print(f'\nQuantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')

Quantization error after 1 iterations: 0.11
Quantization error after 100 iterations: 0.03

Quantization error after 1000 iterations: 0.02


### Plot 23: Increasing SOM size to 25 and LR to .7
Again, decrease in first iteration error.

In [50]:
som_sz = 25
n_features = 4
max_iter = 1

np.random.seed(0)

iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=.9, init_sigma=10.0, verbose=False)
iris_u_map0 = iris_som.u_matrix()
iris_som.fit(iris_x.copy())
iris_u_map = iris_som.u_matrix()
print(f'Quantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')
# print(f'Your initial u-matrix is (before any training):\n{iris_u_map0}')
# print(f'Your u-matrix is (after 1 step):\n{iris_u_map}\n')

max_iter = 100
iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_som.fit(iris_x.copy())
iris_u_map2 = iris_som.u_matrix()
print(f'Quantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')
# print(f'Your u-matrix is (after 100 steps):\n{iris_u_map2}')

max_iter = 1000
iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_som.fit(iris_x.copy())
print(f'\nQuantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')

Quantization error after 1 iterations: 0.10
Quantization error after 100 iterations: 0.03

Quantization error after 1000 iterations: 0.02


### Plot 24: Increasing SOM size to 25 and LR to .9
We are pushing the learning rate high now and no changes over later iterations.

In [51]:
som_sz = 25
n_features = 4
max_iter = 1

np.random.seed(0)

iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=2, init_sigma=10.0, verbose=False)
iris_u_map0 = iris_som.u_matrix()
iris_som.fit(iris_x.copy())
iris_u_map = iris_som.u_matrix()
print(f'Quantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')
# print(f'Your initial u-matrix is (before any training):\n{iris_u_map0}')
# print(f'Your u-matrix is (after 1 step):\n{iris_u_map}\n')

max_iter = 100
iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_som.fit(iris_x.copy())
iris_u_map2 = iris_som.u_matrix()
print(f'Quantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')
# print(f'Your u-matrix is (after 100 steps):\n{iris_u_map2}')

max_iter = 1000
iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_som.fit(iris_x.copy())
print(f'\nQuantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')

Quantization error after 1 iterations: 0.09
Quantization error after 100 iterations: 0.03

Quantization error after 1000 iterations: 0.02


### Plot 25: Increasing SOM size to 25 and LR to 2
Now, we are starting to get absurd with the learning rates and not learning much at later iterations.

In [54]:
som_sz = 25
n_features = 4
max_iter = 1

np.random.seed(0)

iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=20, init_sigma=10.0, verbose=False)
iris_u_map0 = iris_som.u_matrix()
iris_som.fit(iris_x.copy())
iris_u_map = iris_som.u_matrix()
print(f'Quantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')
# print(f'Your initial u-matrix is (before any training):\n{iris_u_map0}')
# print(f'Your u-matrix is (after 1 step):\n{iris_u_map}\n')

max_iter = 100
iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_som.fit(iris_x.copy())
iris_u_map2 = iris_som.u_matrix()
print(f'Quantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')
# print(f'Your u-matrix is (after 100 steps):\n{iris_u_map2}')

max_iter = 1000
iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_som.fit(iris_x.copy())
print(f'\nQuantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')

Quantization error after 1 iterations: 0.15
Quantization error after 100 iterations: 0.03

Quantization error after 1000 iterations: 0.02


### Plot 26: Increasing SOM size to 25 and LR to 20
WE HAVE CHANGES! Our learning rate in our first iteration causes it to go haywire. We likely overshot the first iteration big time and therefore increased our error heavily as we moved datapoints too far. It took an extremely high Learning Rate to do so, but it is possible..
 

## Changing Sigma with Learning Rate and SOM Size. Decreasing First.

In [55]:
som_sz = 5
n_features = 4
max_iter = 1

np.random.seed(0)

iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.15, init_sigma=8.0, verbose=False)
iris_u_map0 = iris_som.u_matrix()
iris_som.fit(iris_x.copy())
iris_u_map = iris_som.u_matrix()
print(f'Quantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')
# print(f'Your initial u-matrix is (before any training):\n{iris_u_map0}')
# print(f'Your u-matrix is (after 1 step):\n{iris_u_map}\n')

max_iter = 100
iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_som.fit(iris_x.copy())
iris_u_map2 = iris_som.u_matrix()
print(f'Quantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')
# print(f'Your u-matrix is (after 100 steps):\n{iris_u_map2}')

max_iter = 1000
iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_som.fit(iris_x.copy())
print(f'\nQuantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')

Quantization error after 1 iterations: 0.24
Quantization error after 100 iterations: 0.08

Quantization error after 1000 iterations: 0.04


### Plot 27: Decreasing SOM size to 5 and LR to .15 and Sigma to 8
Our initial error iterations are [.22,.08,.04]. We have an increase consistent with our learning rate errors. Sigma seems to provide no change here.

In [69]:
som_sz = 3
n_features = 4
max_iter = 1

np.random.seed(0)

iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.1, init_sigma=5.0, verbose=False)
iris_u_map0 = iris_som.u_matrix()
iris_som.fit(iris_x.copy())
iris_u_map = iris_som.u_matrix()
print(f'Quantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')
# print(f'Your initial u-matrix is (before any training):\n{iris_u_map0}')
# print(f'Your u-matrix is (after 1 step):\n{iris_u_map}\n')

max_iter = 100
iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_som.fit(iris_x.copy())
iris_u_map2 = iris_som.u_matrix()
print(f'Quantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')
# print(f'Your u-matrix is (after 100 steps):\n{iris_u_map2}')

max_iter = 1000
iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_som.fit(iris_x.copy())
print(f'\nQuantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')

Quantization error after 1 iterations: 0.25
Quantization error after 100 iterations: 0.13

Quantization error after 1000 iterations: 0.09


### Plot 28: Decreasing SOM size to 3 and LR to .1 and Sigma to 5
Our first plot at SOM 3 LR 2 and Sigma 10 is: [.23, .13, .09]. This has the same increase of our first iteration in the Learning Rate increase, which is consistent here. Sigma doesn't seem to cause any changes so far.

In [70]:
som_sz = 2
n_features = 4
max_iter = 1

np.random.seed(0)

iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.08, init_sigma=3.0, verbose=False)
iris_u_map0 = iris_som.u_matrix()
iris_som.fit(iris_x.copy())
iris_u_map = iris_som.u_matrix()
print(f'Quantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')
# print(f'Your initial u-matrix is (before any training):\n{iris_u_map0}')
# print(f'Your u-matrix is (after 1 step):\n{iris_u_map}\n')

max_iter = 100
iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_som.fit(iris_x.copy())
iris_u_map2 = iris_som.u_matrix()
print(f'Quantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')
# print(f'Your u-matrix is (after 100 steps):\n{iris_u_map2}')

max_iter = 1000
iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_som.fit(iris_x.copy())
print(f'\nQuantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')

Quantization error after 1 iterations: 0.37
Quantization error after 100 iterations: 0.17

Quantization error after 1000 iterations: 0.16


### Plot 29: Decreasing SOM size to 2 and LR to .08 and Sigma to 3
This is the same error cascade as our learning rate increase without the Sigma change. Our original one was: [.34, .17, .16]. Again, no influence from Sigma.

In [71]:
som_sz = 2
n_features = 4
max_iter = 1

np.random.seed(0)

iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.02, init_sigma=1.0, verbose=False)
iris_u_map0 = iris_som.u_matrix()
iris_som.fit(iris_x.copy())
iris_u_map = iris_som.u_matrix()
print(f'Quantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')
# print(f'Your initial u-matrix is (before any training):\n{iris_u_map0}')
# print(f'Your u-matrix is (after 1 step):\n{iris_u_map}\n')

max_iter = 100
iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_som.fit(iris_x.copy())
iris_u_map2 = iris_som.u_matrix()
print(f'Quantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')
# print(f'Your u-matrix is (after 100 steps):\n{iris_u_map2}')

max_iter = 1000
iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_som.fit(iris_x.copy())
print(f'\nQuantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')

Quantization error after 1 iterations: 0.38
Quantization error after 100 iterations: 0.17

Quantization error after 1000 iterations: 0.16


### Plot 30: Decreasing SOM size to 2 and LR to .02 and Sigma to 1
This is the same increase in error rate with the first iteration. Again, Sigma changes nothing.

In [72]:
som_sz = 2
n_features = 4
max_iter = 1

np.random.seed(0)

iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.001, init_sigma=.01, verbose=False)
iris_u_map0 = iris_som.u_matrix()
iris_som.fit(iris_x.copy())
iris_u_map = iris_som.u_matrix()
print(f'Quantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')
# print(f'Your initial u-matrix is (before any training):\n{iris_u_map0}')
# print(f'Your u-matrix is (after 1 step):\n{iris_u_map}\n')

max_iter = 100
iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_som.fit(iris_x.copy())
iris_u_map2 = iris_som.u_matrix()
print(f'Quantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')
# print(f'Your u-matrix is (after 100 steps):\n{iris_u_map2}')

max_iter = 1000
iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_som.fit(iris_x.copy())
print(f'\nQuantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')

Quantization error after 1 iterations: 0.39
Quantization error after 100 iterations: 0.17

Quantization error after 1000 iterations: 0.16


### Plot 31: Decreasing SOM size to 2 and LR to .001 and Sigma to 0.01
There is only an increase in the first iteration error.

## Increasing LR, Sigma and SOM SZ

In [73]:
som_sz = 9
n_features = 4
max_iter = 1

np.random.seed(0)

iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.25, init_sigma=12.0, verbose=False)
iris_u_map0 = iris_som.u_matrix()
iris_som.fit(iris_x.copy())
iris_u_map = iris_som.u_matrix()
print(f'Quantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')
# print(f'Your initial u-matrix is (before any training):\n{iris_u_map0}')
# print(f'Your u-matrix is (after 1 step):\n{iris_u_map}\n')

max_iter = 100
iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_som.fit(iris_x.copy())
iris_u_map2 = iris_som.u_matrix()
print(f'Quantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')
# print(f'Your u-matrix is (after 100 steps):\n{iris_u_map2}')

max_iter = 1000
iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_som.fit(iris_x.copy())
print(f'\nQuantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')

Quantization error after 1 iterations: 0.13
Quantization error after 100 iterations: 0.05

Quantization error after 1000 iterations: 0.03


### Plot 32: Increasing SOM size to 9 and LR to .25 and Sigma to 12
We have the same error rate as with the above vanilla SOM Sz=9 and LR=.2. Our original was: [.14,.05,.03]. Again, not seeing so many changes with Sigma.

In [78]:
som_sz = 20
n_features = 4
max_iter = 1

np.random.seed(0)

iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.35, init_sigma=18.0, verbose=False)
iris_u_map0 = iris_som.u_matrix()
iris_som.fit(iris_x.copy())
iris_u_map = iris_som.u_matrix()
print(f'Quantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')
# print(f'Your initial u-matrix is (before any training):\n{iris_u_map0}')
# print(f'Your u-matrix is (after 1 step):\n{iris_u_map}\n')

max_iter = 100
iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_som.fit(iris_x.copy())
iris_u_map2 = iris_som.u_matrix()
print(f'Quantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')
# print(f'Your u-matrix is (after 100 steps):\n{iris_u_map2}')

max_iter = 1000
iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_som.fit(iris_x.copy())
print(f'\nQuantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')

Quantization error after 1 iterations: 0.13
Quantization error after 100 iterations: 0.03

Quantization error after 1000 iterations: 0.02


### Plot 33: Increasing SOM size to 20 and LR to .35 and Sigma to 18
Our vanilla had: [.15, .03, .02]. We now have the exact same as with no change to Sigma.

In [79]:
som_sz = 500
n_features = 4
max_iter = 1

np.random.seed(0)

iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=2, init_sigma=50.0, verbose=False)
iris_u_map0 = iris_som.u_matrix()
iris_som.fit(iris_x.copy())
iris_u_map = iris_som.u_matrix()
print(f'Quantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')
# print(f'Your initial u-matrix is (before any training):\n{iris_u_map0}')
# print(f'Your u-matrix is (after 1 step):\n{iris_u_map}\n')

max_iter = 100
iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_som.fit(iris_x.copy())
iris_u_map2 = iris_som.u_matrix()
print(f'Quantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')
# print(f'Your u-matrix is (after 100 steps):\n{iris_u_map2}')

max_iter = 1000
iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_som.fit(iris_x.copy())
print(f'\nQuantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')

Quantization error after 1 iterations: 0.02
Quantization error after 100 iterations: 0.02

Quantization error after 1000 iterations: 0.01


### Plot 34: Increasing SOM size to 500 and LR to 2 and Sigma to 50
Another Big Kahuna, another no change. Sigma isn't changing us here.

## Increasing Sigma with SOM SZ = 25 and LR = .25

In [96]:
som_sz = 25
n_features = 4
max_iter = 1

np.random.seed(0)

iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.25, init_sigma=10.0, verbose=False)
iris_u_map0 = iris_som.u_matrix()
iris_som.fit(iris_x.copy())
iris_u_map = iris_som.u_matrix()
print(f'Quantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')
# print(f'Your initial u-matrix is (before any training):\n{iris_u_map0}')
# print(f'Your u-matrix is (after 1 step):\n{iris_u_map}\n')

max_iter = 100
iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_som.fit(iris_x.copy())
iris_u_map2 = iris_som.u_matrix()
print(f'Quantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')
# print(f'Your u-matrix is (after 100 steps):\n{iris_u_map2}')

max_iter = 1000
iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_som.fit(iris_x.copy())
print(f'\nQuantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')

Quantization error after 1 iterations: 0.15
Quantization error after 100 iterations: 0.03

Quantization error after 1000 iterations: 0.02


### Plot 35: Baseline. [.15, .03, .02].

In [80]:
som_sz = 25
n_features = 4
max_iter = 1

np.random.seed(0)

iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.25, init_sigma=15.0, verbose=False)
iris_u_map0 = iris_som.u_matrix()
iris_som.fit(iris_x.copy())
iris_u_map = iris_som.u_matrix()
print(f'Quantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')
# print(f'Your initial u-matrix is (before any training):\n{iris_u_map0}')
# print(f'Your u-matrix is (after 1 step):\n{iris_u_map}\n')

max_iter = 100
iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_som.fit(iris_x.copy())
iris_u_map2 = iris_som.u_matrix()
print(f'Quantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')
# print(f'Your u-matrix is (after 100 steps):\n{iris_u_map2}')

max_iter = 1000
iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_som.fit(iris_x.copy())
print(f'\nQuantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')

Quantization error after 1 iterations: 0.14
Quantization error after 100 iterations: 0.03

Quantization error after 1000 iterations: 0.02


### Plot 36: Increasing SOM size to 25 and LR to .25 and Sigma to 15
We see a first iteration change again! This constitutes a decrease overall in the first iteration. 

In [81]:
som_sz = 25
n_features = 4
max_iter = 1

np.random.seed(0)

iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.25, init_sigma=25.0, verbose=False)
iris_u_map0 = iris_som.u_matrix()
iris_som.fit(iris_x.copy())
iris_u_map = iris_som.u_matrix()
print(f'Quantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')
# print(f'Your initial u-matrix is (before any training):\n{iris_u_map0}')
# print(f'Your u-matrix is (after 1 step):\n{iris_u_map}\n')

max_iter = 100
iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_som.fit(iris_x.copy())
iris_u_map2 = iris_som.u_matrix()
print(f'Quantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')
# print(f'Your u-matrix is (after 100 steps):\n{iris_u_map2}')

max_iter = 1000
iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_som.fit(iris_x.copy())
print(f'\nQuantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')

Quantization error after 1 iterations: 0.14
Quantization error after 100 iterations: 0.03

Quantization error after 1000 iterations: 0.02


### Plot 37: Increasing SOM size to 25 and LR to .25 and Sigma to 25
This is no change from Sigma. It is becoming apparent Sigma has little influence.

In [83]:
som_sz = 25
n_features = 4
max_iter = 1

np.random.seed(0)

iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.25, init_sigma=50.0, verbose=False)
iris_u_map0 = iris_som.u_matrix()
iris_som.fit(iris_x.copy())
iris_u_map = iris_som.u_matrix()
print(f'Quantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')
# print(f'Your initial u-matrix is (before any training):\n{iris_u_map0}')
# print(f'Your u-matrix is (after 1 step):\n{iris_u_map}\n')

max_iter = 100
iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_som.fit(iris_x.copy())
iris_u_map2 = iris_som.u_matrix()
print(f'Quantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')
# print(f'Your u-matrix is (after 100 steps):\n{iris_u_map2}')

max_iter = 1000
iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_som.fit(iris_x.copy())
print(f'\nQuantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')

Quantization error after 1 iterations: 0.14
Quantization error after 100 iterations: 0.03

Quantization error after 1000 iterations: 0.02


### Plot 38: Increasing SOM size to 25 and LR to .25 and Sigma to 50
We have an increase in Sigma to 50 here and again no change from earlier Sigmas.

In [84]:
som_sz = 25
n_features = 4
max_iter = 1

np.random.seed(0)

iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.25, init_sigma=500.0, verbose=False)
iris_u_map0 = iris_som.u_matrix()
iris_som.fit(iris_x.copy())
iris_u_map = iris_som.u_matrix()
print(f'Quantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')
# print(f'Your initial u-matrix is (before any training):\n{iris_u_map0}')
# print(f'Your u-matrix is (after 1 step):\n{iris_u_map}\n')

max_iter = 100
iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_som.fit(iris_x.copy())
iris_u_map2 = iris_som.u_matrix()
print(f'Quantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')
# print(f'Your u-matrix is (after 100 steps):\n{iris_u_map2}')

max_iter = 1000
iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_som.fit(iris_x.copy())
print(f'\nQuantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')

Quantization error after 1 iterations: 0.14
Quantization error after 100 iterations: 0.03

Quantization error after 1000 iterations: 0.02


### Plot 39: Increasing SOM size to 25 and LR to .25 and Sigma to 500
A Sigma Big Kahuna and we get nowhere! Sigma at 500 provides no change. This is likely because of decay rate providing most of the difference, or Sigma has little overall influence on error.

In [85]:
som_sz = 25
n_features = 4
max_iter = 1

np.random.seed(0)

iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.25, init_sigma=7.0, verbose=False)
iris_u_map0 = iris_som.u_matrix()
iris_som.fit(iris_x.copy())
iris_u_map = iris_som.u_matrix()
print(f'Quantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')
# print(f'Your initial u-matrix is (before any training):\n{iris_u_map0}')
# print(f'Your u-matrix is (after 1 step):\n{iris_u_map}\n')

max_iter = 100
iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_som.fit(iris_x.copy())
iris_u_map2 = iris_som.u_matrix()
print(f'Quantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')
# print(f'Your u-matrix is (after 100 steps):\n{iris_u_map2}')

max_iter = 1000
iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_som.fit(iris_x.copy())
print(f'\nQuantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')

Quantization error after 1 iterations: 0.16
Quantization error after 100 iterations: 0.03

Quantization error after 1000 iterations: 0.02


### Plot 40: Decreasing SOM size to 25 and LR to .25 and Sigma to 7
This actually increases our error slightly for the first iteration but has no change on later iterations.

In [87]:
som_sz = 25
n_features = 4
max_iter = 1

np.random.seed(0)

iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.25, init_sigma=5.0, verbose=False)
iris_u_map0 = iris_som.u_matrix()
iris_som.fit(iris_x.copy())
iris_u_map = iris_som.u_matrix()
print(f'Quantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')
# print(f'Your initial u-matrix is (before any training):\n{iris_u_map0}')
# print(f'Your u-matrix is (after 1 step):\n{iris_u_map}\n')

max_iter = 100
iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_som.fit(iris_x.copy())
iris_u_map2 = iris_som.u_matrix()
print(f'Quantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')
# print(f'Your u-matrix is (after 100 steps):\n{iris_u_map2}')

max_iter = 1000
iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_som.fit(iris_x.copy())
print(f'\nQuantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')

Quantization error after 1 iterations: 0.16
Quantization error after 100 iterations: 0.03

Quantization error after 1000 iterations: 0.02


### Plot 41: Decreasing SOM size to 25 and LR to .25 and Sigma to 5
This provides no change again, Sigma maintaining the same line.

In [88]:
som_sz = 25
n_features = 4
max_iter = 1

np.random.seed(0)

iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.25, init_sigma=2.0, verbose=False)
iris_u_map0 = iris_som.u_matrix()
iris_som.fit(iris_x.copy())
iris_u_map = iris_som.u_matrix()
print(f'Quantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')
# print(f'Your initial u-matrix is (before any training):\n{iris_u_map0}')
# print(f'Your u-matrix is (after 1 step):\n{iris_u_map}\n')

max_iter = 100
iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_som.fit(iris_x.copy())
iris_u_map2 = iris_som.u_matrix()
print(f'Quantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')
# print(f'Your u-matrix is (after 100 steps):\n{iris_u_map2}')

max_iter = 1000
iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_som.fit(iris_x.copy())
print(f'\nQuantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')

Quantization error after 1 iterations: 0.16
Quantization error after 100 iterations: 0.03

Quantization error after 1000 iterations: 0.02


### Plot 42: Decreasing SOM size to 25 and LR to .25 and Sigma to 2
Again, Sigma changes nothing.

In [89]:
som_sz = 25
n_features = 4
max_iter = 1

np.random.seed(0)

iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.25, init_sigma=.8, verbose=False)
iris_u_map0 = iris_som.u_matrix()
iris_som.fit(iris_x.copy())
iris_u_map = iris_som.u_matrix()
print(f'Quantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')
# print(f'Your initial u-matrix is (before any training):\n{iris_u_map0}')
# print(f'Your u-matrix is (after 1 step):\n{iris_u_map}\n')

max_iter = 100
iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_som.fit(iris_x.copy())
iris_u_map2 = iris_som.u_matrix()
print(f'Quantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')
# print(f'Your u-matrix is (after 100 steps):\n{iris_u_map2}')

max_iter = 1000
iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_som.fit(iris_x.copy())
print(f'\nQuantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')

Quantization error after 1 iterations: 0.16
Quantization error after 100 iterations: 0.03

Quantization error after 1000 iterations: 0.02


### Plot 43: Decreasing SOM size to 25 and LR to .25 and Sigma to 0.8
Same as above.

In [90]:
som_sz = 25
n_features = 4
max_iter = 1

np.random.seed(0)

iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.25, init_sigma=.05, verbose=False)
iris_u_map0 = iris_som.u_matrix()
iris_som.fit(iris_x.copy())
iris_u_map = iris_som.u_matrix()
print(f'Quantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')
# print(f'Your initial u-matrix is (before any training):\n{iris_u_map0}')
# print(f'Your u-matrix is (after 1 step):\n{iris_u_map}\n')

max_iter = 100
iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_som.fit(iris_x.copy())
iris_u_map2 = iris_som.u_matrix()
print(f'Quantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')
# print(f'Your u-matrix is (after 100 steps):\n{iris_u_map2}')

max_iter = 1000
iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_som.fit(iris_x.copy())
print(f'\nQuantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')

Quantization error after 1 iterations: 0.16
Quantization error after 100 iterations: 0.03

Quantization error after 1000 iterations: 0.02


### Plot 44: Decreasing SOM size to 25 and LR to .25 and Sigma to 0.05
Our Sigma is super small and we had no difference.

In [91]:
som_sz = 25
n_features = 4
max_iter = 1

np.random.seed(0)

iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.25, init_sigma=.0005, verbose=False)
iris_u_map0 = iris_som.u_matrix()
iris_som.fit(iris_x.copy())
iris_u_map = iris_som.u_matrix()
print(f'Quantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')
# print(f'Your initial u-matrix is (before any training):\n{iris_u_map0}')
# print(f'Your u-matrix is (after 1 step):\n{iris_u_map}\n')

max_iter = 100
iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_som.fit(iris_x.copy())
iris_u_map2 = iris_som.u_matrix()
print(f'Quantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')
# print(f'Your u-matrix is (after 100 steps):\n{iris_u_map2}')

max_iter = 1000
iris_som = som.SOM(som_sz, n_features, max_iter, init_lr=0.2, init_sigma=10.0, verbose=False)
iris_som.fit(iris_x.copy())
print(f'\nQuantization error after {max_iter} iterations: {iris_som.error(iris_x):.2f}')

Quantization error after 1 iterations: 0.16
Quantization error after 100 iterations: 0.03

Quantization error after 1000 iterations: 0.02


### Plot 45: Decreasing SOM size to 25 and LR to .25 and Sigma to 0.0005
Even with a tiny Sigma, we receive no difference. 